# A simple RAG application using open-source models (Ollama/lmStudio) or OpenAI

### Note: Given that the LmStudio API calls are designed to work with the OpenAI SDK, we’re able to integrate the OpenAI SDK into LmStudio

In [37]:
# TYPE = "Ollama"
# MODEL = "gemma2:2b"


# TYPE = "OpenAI"
# MODEL = "gpt-3.5-turbo"
# OPENAI_KEY=""


TYPE= "LmStudio"
MODEL = "gemma-3-4b-it"
EMBED_MODEL = "nomic-embed-text-v1.5"

In [38]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from openai import OpenAI
from typing import List

if TYPE == "OpenAI":
    model = ChatOpenAI(
        openai_api_key=OPENAI_KEY,
        model=MODEL
    )
    embeddings = OpenAIEmbeddings()
elif TYPE == "Ollama":
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)
elif TYPE == "LmStudio":
    model = ChatOpenAI(
        openai_api_key="lm-studio",  # Can be any non-empty string for LM Studio
        model=MODEL,
        base_url="http://localhost:1234/v1"  # Default LM Studio API endpoint
    )

    # Embeding for LmStudio
    class LMSEmbeddingFunction:
        def __init__(self):
            self.client = OpenAI(
                base_url="http://localhost:1234/v1",
                api_key="lm-studio"
            )
        
        def embed_documents(self, texts: List[str], model=EMBED_MODEL) -> List[List[float]]:
            embeddings = self.client.embeddings.create(input=texts, model=model).data
            return [e.embedding for e in embeddings]
        
        def embed_query(self, text: str, model=EMBED_MODEL) -> List[float]:
            embedding = self.client.embeddings.create(input=[text], model=model).data[0].embedding
            return embedding

    # Create the embedding function
    embeddings = LMSEmbeddingFunction()
else:
    raise ValueError(f"Invalid Model Selection: {TYPE}")

# Only invoke if model is properly initialized
if model is not None:
    response = model.invoke("Tell me a joke")
    print(response)
else:
    print("Model not initialized")

content="Why don't scientists trust atoms? \n\n... Because they make up everything! 😄 \n\n---\n\nWould you like to hear another one?" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 13, 'total_tokens': 43, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gemma-3-4b-it', 'system_fingerprint': 'gemma-3-4b-it', 'id': 'chatcmpl-i4qfvd1buil7da2vke1ug8', 'finish_reason': 'stop', 'logprobs': None} id='run-da364829-f597-4964-9953-7f326a01d3a9-0' usage_metadata={'input_tokens': 13, 'output_tokens': 30, 'total_tokens': 43, 'input_token_details': {}, 'output_token_details': {}}


In [39]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
chain.invoke("Tell me a joke")

"Why don't scientists trust atoms? \n\n... Because they make up everything! 😄 \n\n---\n\nWould you like to hear another one?"

In [40]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [41]:
chain = prompt | model | parser

chain.invoke({"context": "My parents named me Santiago", "question": "What's your name'?"})

'Santiago'

In [42]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data\\NikiCV.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'producer': 'cairo 1.18.0 (https://cairographics.org)', 'creator': 'Mozilla Firefox 136.0.2', 'creationdate': '2025-03-26T10:09:46+03:30', 'source': 'data\\NikiCV.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='M MO OH HA AM MM MA AD D  N NI IK KR RA AV VE ES SH H\nB B. .S Sc c. .\nC Co om mp pu ut te er r  S Sc ci ie en nc ce e\nU Un ni iv ve er rs si it ty y  o of f  S Se em mn na an n\n\uf073 S Se ep p  2 20 01 12 2  - -  N No ov v  2 20 01 17 7\n\uf3c5 S Se em mn na an n, ,  I Ir ra an n\nR Re el le ev va an nt t  C Co ou ur rs se es s\n- Data Structures\n- Algorithm Design\n- Database Management Systems\n- Computer Vision\n- Software Design Methodology\n• L Li in nu ux x  | |  D Do oc ck ke er r  | |  G Gi it t  , ,\n• N No od de e. .j js s  | |  J Ja av va aS Sc cr ri ip pt t  , ,\n• G Go oL La an ng g  , ,\n• P Py yt th ho on n  , ,\n• U Un ni it ty y3 3D D  ( (C C# #) )  , ,\n• N No oS SQ QL L  | |  S SQ QL L  , ,\n• C CI I/ /C CD D  | |

In [43]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [44]:
retriever = vectorstore.as_retriever()
retriever.invoke("what is this pdf?")

[Document(metadata={'producer': 'cairo 1.18.0 (https://cairographics.org)', 'creator': 'Mozilla Firefox 136.0.2', 'creationdate': '2025-03-26T10:09:46+03:30', 'source': 'data\\NikiCV.pdf', 'total_pages': 2, 'page': 1, 'page_label': '2'}, page_content='\uf111 D Da ad de eh h  G Go os st ta ar r \ue55f \ue55fS Se em mn na an n, ,  I Ir ra an n\nNode.js Linux Docker Git Redis MySQL MongoDB Angular2+ Ionic2+ WikiMedia\nS SO OF FT TW WA AR RE E  E EN NG GI IN NE EE ER R  ((B BA AC CK K- -E EN ND D  D DE EV VE EL LO OP PE ERR) ) Apr 2015 - Oct 2015\n• Back-end developer for several web and mobile\napplications.\n• Ported the "AMLAKNET" project to ASP MVC 3.\n• Designed and developed a cross-platform application called "TOUCHSI" for booking\ncars.\n• Wrote sample code and library for porting Java and Android push noti�cation.\nGitLab ASP MVC MSSQL Angular2+ Ionic2+ Windows Server JavaScript HTML CSS'),
 Document(metadata={'producer': 'cairo 1.18.0 (https://cairographics.org)', 'creator': 'Moz

In [45]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [46]:
questions = [
    "What is the purpose of the pdf?",
    # "how many pages is in this pdf?",
    # "How many coding assignments are there in the program?",
    # "Is there a program certificate upon completion?",
    # "What programming language will be used in the pdf?",
    "tell me a breif about it?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What is the purpose of the pdf?
Answer: Based on the provided document metadata and page content, the PDF appears to be a Curriculum Vitae (CV) or resume for someone named Hootan. It details their education, skills, work experience, and contact information.

Question: tell me a breif about it?
Answer: This document is a resume/CV for Hootan Azimi, a freelance software developer. He has experience as a back-end developer across various projects, including CRM development, product deployments, and cross-platform application design. His skills encompass technologies like Node.js, Angular2+, React Native, GoLang, C#, Linux, Docker, MongoDB, and more. He’s also worked on personal projects such as game development (Unity3D) and a static website generator (Hugo).



In [47]:
chain.batch([{"question": q} for q in questions])

['Based on the provided document metadata and page content, the PDF appears to be a Curriculum Vitae (CV) or resume for someone named Hootan. It details their education, skills, work experience, and contact information.',
 'This document is a resume/CV for Hootan Azimi, a freelance software developer. He has experience as a back-end developer working on various projects including CRM systems, web applications (like "TOUCHSI" and "KingStone"), mobile apps (using React Native and Unity3D), and IoT projects. His skills include Node.js, GoLang, C#, JavaScript, Linux, Docker, MongoDB, Git, and several front-end technologies like Angular2+, Ionic2+, HTML, and CSS. He has a computer science degree from the University of Semnan and has worked on personal projects alongside his freelance work.']

In [48]:
for s in chain.stream({"question": "What is the purpose of the pdf?"}):
    print(s, end="", flush=True)

Based on the provided document metadata and page content, the PDF appears to be a Curriculum Vitae (CV) or resume for someone named Hootan. It details their education, skills, work experience, and contact information.

In [109]:
# from langchain_community.document_loaders import WebBaseLoader
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# loader = WebBaseLoader("https://www.ml.school")
# docs = loader.load()
# documents = RecursiveCharacterTextSplitter(
#     chunk_size=1000, chunk_overlap=200
# ).split_documents(docs)

# documents

[Document(page_content='Building Machine Learning Systems That Don\'t Suck"This is the best machine learning course I\'ve done ever. Worth every cent."Jose Reyes, AI/ML at Cevo AustraliaBuilding Machine Learning Systems That Don\'t SuckA live, interactive program that\'ll help you build production-ready machine learning systems from the ground up.Next cohort:\xa0April 8 - 25, 2024Check the schedule for more details about upcoming cohorts.Register nowLearn how to design, build, deploy, and scale machine learning systems to solve real-world problems.I\'ll lose my mind if I see another book or course teaching people the same basic ideas for the hundredth time. Most people are stuck in beginner mode, and finding help to solve real-world problems is hard.I want to change that.I started writing software 30 years ago. I\'ve written pipelines and trained models for some of the largest companies in the world. I want to show you how to do the same.This is the class I wish I had taken when I star